In [1]:
import pandas as pd
from utils import save_csv

to_your_path = '/data/lujd/'
datapath = to_your_path + "TCRdata/collect/"

filename = "main_dataset_tcr_phla_cleaned.csv"
merge_df = pd.read_csv(datapath+filename, sep=',')
merge_df = merge_df[merge_df['mhc.class']=='MHCI']
print(len(merge_df))

filename = "MIRA.csv"
mira_df = pd.read_csv(datapath+filename, sep=',')
mira_df = mira_df[mira_df['mhc.class']=='MHCI']
mira_df['dataset.tag'] = "mira"
print(len(mira_df))

savepath = to_your_path + "TCRdata/final/"

174673
53613


---

### Level-I

In [2]:
merge_df_l1 = merge_df[['antigen.epitope', 'beta.cdr3', 'dataset.tag']].drop_duplicates(
    subset=['antigen.epitope', 'beta.cdr3'], ignore_index=True, keep='first')
print(len(merge_df_l1))
mira_df_l1 = mira_df[['antigen.epitope', 'beta.cdr3', 'dataset.tag']].drop_duplicates(
    subset=['antigen.epitope', 'beta.cdr3'], ignore_index=True, keep='first')
print(len(mira_df_l1))

165329
49326


In [3]:
level1_df = pd.concat([merge_df_l1, mira_df_l1], axis=0).drop_duplicates(
    subset=['antigen.epitope', 'beta.cdr3'], ignore_index=True, keep='first')
print(f"number of duplicated data: {len(level1_df)}")

level1_df = level1_df[level1_df['antigen.epitope']!='(UNK)']
level1_df = level1_df[level1_df['beta.cdr3']!='(UNK)']
print(f"number of dropna data: {len(level1_df)}")

savename = "Hi-TpH-level-I.csv"
save_csv(level1_df, savepath, savename)

number of duplicated data: 214641
number of dropna data: 214641
File /data/lujd/TCRdata/final/Hi-TpH-level-I.csv has been successfully saved.


In [4]:
print(f"samples: {len(level1_df)}")
print(f"TCRs:    {level1_df['beta.cdr3'].nunique()}")
print(f"peptides:{level1_df['antigen.epitope'].nunique()}")

samples: 214641
TCRs:    204376
peptides:1647


In [5]:
for tag in level1_df['dataset.tag'].unique():
    tag_df = level1_df[level1_df['dataset.tag']==tag]
    print(tag,
          len(tag_df),
          tag_df['antigen.epitope'].nunique(),
          tag_df['beta.cdr3'].nunique(),
          )

iedb 120289 647 114835
vdjdb 38946 1006 36435
mcpas 6094 278 5967
mira 49312 148 48540


In [11]:
214641/66471, 1647/1391, 204376/61159

(3.22909238615336, 1.184040258806614, 3.341715855393319)

---

### Level-II

- pep hla cdr3b

In [ ]:
level2_df = merge_df[['antigen.epitope', 'hla.allele', 'beta.cdr3', 'dataset.tag']].drop_duplicates(
    subset=['antigen.epitope', 'hla.allele', 'beta.cdr3'], ignore_index=True, keep='first')
print(len(level2_df))

level2_df = level2_df[level2_df['antigen.epitope']!='(UNK)']
level2_df = level2_df[level2_df['beta.cdr3']!='(UNK)']
level2_df = level2_df.reset_index(drop=True)
print(f"number of dropna(pep,cdr3b) data: {len(level2_df)}")

level2_df = level2_df[level2_df['hla.allele']!='(UNK)']
level2_df = level2_df.reset_index(drop=True)
print(f"number of dropna(hla) data: {len(level2_df)}")

print(f"samples: {len(level2_df)}")
print(f"TCRs:    {len(level2_df.drop_duplicates(subset=['beta.cdr3']))}")
print(f"peptides:{level2_df['antigen.epitope'].nunique()}")

167298
number of dropna(pep,cdr3b) data: 167298
number of dropna(hla) data: 80628
samples: 80628
TCRs:    73707
peptides:1403


In [ ]:
target_hla_list = level2_df['hla.allele'].unique().tolist()
print(len(target_hla_list))
target_hla_list[:5]

118


['HLA-E*01:01', 'HLA-E*01:03', 'HLA-B*08', 'HLA-B*44:05', 'HLA-B*08:01']

- read from allele2seq dict

In [22]:
import os
import json
dict_savepath = to_your_path + "TCRdata/collect"
dict_savename = "hla_allele2seq.json"
with open(os.path.join(dict_savepath, dict_savename), "r") as f:
    allele2seq_dict = json.load(f)

all_allele_list = list(allele2seq_dict.keys())     # sorted
print(len(all_allele_list))
all_allele_list[:5]

24153


['HLA-A*01:01:01:01',
 'HLA-A*01:01:01:02N',
 'HLA-A*01:01:01:03',
 'HLA-A*01:01:01:04',
 'HLA-A*01:01:01:05']

In [23]:
short_position = [7,9,24,45,59,62,63,66,67,69,70,73,74,76,77,80,81,84,95,97,99,
                114,116,118,143,147,150,152,156,158,159,163,167,171]        # netmhcpan
short_position = [pos-1 for pos in short_position]

seq_dict = {}
for target_hla in target_hla_list:
    if len(target_hla.split(' ')) > 1:
        target_allele = target_hla.split(' ')[0]
        mutant_pos = int(target_hla.split(' ')[1][1:-1])
        wild_res = target_hla.split(' ')[1][0]
        muta_res = target_hla.split(' ')[1][-1]
        print(target_hla, mutant_pos, wild_res, muta_res)
    else:
        target_allele = target_hla.split(' ')[0]
        mutant_pos = -1
        wild_res = 'X'
        muta_res = 'X'
    for allele in all_allele_list:
        info = {}
        if target_allele in allele:
            if (allele2seq_dict[allele]['clip sequence length'] == 0) | (allele2seq_dict[allele]['short sequence length'] == 0):
                print(allele)
            else:
                if mutant_pos >= 0:
                    if allele2seq_dict[allele]['clip sequence'][mutant_pos-1] != wild_res:
                        print(allele, 1)
                    else:
                        info['source allele'] = allele
                        # reconstruct clip seq
                        info['clip sequence'] = \
                            allele2seq_dict[allele]['clip sequence'][:mutant_pos-1] + muta_res + \
                                allele2seq_dict[allele]['clip sequence'][mutant_pos:]
                        info['clip sequence length'] = len(info['clip sequence'])
                        assert info['clip sequence length']==allele2seq_dict[allele]['clip sequence length']
                        # reconstruct full seq
                        clip_start = allele2seq_dict[allele]['full sequence'].find(allele2seq_dict[allele]['clip sequence'])
                        clip_end = clip_start + len(info['clip sequence'])
                        info['full sequence'] = \
                            allele2seq_dict[allele]['full sequence'][:clip_start] + \
                            info['clip sequence'] + \
                            allele2seq_dict[allele]['full sequence'][clip_end:]
                        info['full sequence length'] = len(info['full sequence'])
                        assert info['full sequence length']==allele2seq_dict[allele]['full sequence length']
                        # reconstruct short seq
                        short_seq = ""
                        for ind in short_position:
                            short_seq += info['clip sequence'][ind]
                        info['short sequence'] = short_seq
                        info['short sequence length'] = len(short_seq)
                        if short_seq != allele2seq_dict[allele]['short sequence']:
                            print(allele2seq_dict[allele]['short sequence'], short_seq)
                        seq_dict[target_hla] = info
                else:
                    info['source allele'] = allele
                    info['full sequence'] = allele2seq_dict[allele]['full sequence']
                    info['full sequence length'] = allele2seq_dict[allele]['full sequence length']
                    info['clip sequence'] = allele2seq_dict[allele]['clip sequence']
                    info['clip sequence length'] = allele2seq_dict[allele]['clip sequence length']
                    info['short sequence'] = allele2seq_dict[allele]['short sequence']
                    info['short sequence length'] = allele2seq_dict[allele]['short sequence length']
                    seq_dict[target_hla] = info
            break

HLA-A*02:01 K66A 66 K A
YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY YFAMYGEAVAHTHVDTLYVRYHYYTWAVLAYTWY
HLA-A*02:01 E63Q 63 E Q
YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY YFAMYGQKVAHTHVDTLYVRYHYYTWAVLAYTWY
HLA-A*02:01 A150P 150 A P
YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY YFAMYGEKVAHTHVDTLYVRYHYYTWPVLAYTWY
HLA-B*35:08 Q65A 65 Q A
HLA-B*35:08 T69A 69 T A
YYATYRNIFTNTYESNLYIRYDSYTWAVRAYLWY YYATYRNIFANTYESNLYIRYDSYTWAVRAYLWY
HLA-B*35:08 Q155A 155 Q A
HLA-A*02:01 A69G 69 A G
YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY YFAMYGEKVGHTHVDTLYVRYHYYTWAVLAYTWY
HLA-A*02:01 E166A 166 E A
HLA-A*02:01 R65A 65 R A
HLA-A*02:01 T163A 163 T A
YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYAWY
HLA-A*02:01 W167A 167 W A
YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTAY


In [24]:
savename = "Hi-TpH-hla_allele2seq.json"

with open(os.path.join(savepath, savename), "w") as f:
    json.dump(seq_dict, f, indent=4)

print(F"File {os.path.join(savepath, savename)} has been successfully saved.")

File /data/lujd/TCRdata/final/Hi-TpH-hla_allele2seq.json has been successfully saved.


In [ ]:
print(set(target_hla_list) - set(seq_dict.keys()))
print(len(level2_df[level2_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]))

level2_df = level2_df[~level2_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]
level2_df = level2_df.reset_index(drop=True)
print(f"number of hla seq data: {len(level2_df)}")

{'HLA-B*12', 'HLA-A*24:01', 'HLA-A*08:01'}
161
number of hla seq data: 80467


In [ ]:
level2_df['hla.full.seq'] = level2_df['hla.allele'].apply(lambda x: seq_dict[x]['full sequence'])
level2_df['hla.clip.seq'] = level2_df['hla.allele'].apply(lambda x: seq_dict[x]['clip sequence'])
level2_df['hla.short.seq'] = level2_df['hla.allele'].apply(lambda x: seq_dict[x]['short sequence'])

level2_df = level2_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['beta.cdr3', 'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(level2_df))

print(len(level2_df.drop_duplicates(subset=['beta.cdr3', 'antigen.epitope', 'hla.full.seq'])))
print(len(level2_df.drop_duplicates(subset=['beta.cdr3', 'antigen.epitope', 'hla.clip.seq'])))
print(len(level2_df.drop_duplicates(subset=['beta.cdr3', 'antigen.epitope', 'hla.short.seq'])))

78679
78679
78672
78665


In [ ]:
for c in level2_df:
    print(len(level2_df[level2_df[c]=="(UNK)"]))

0
0
0
0
0
0
0


In [ ]:
savename = "Hi-TpH-level-II.csv"
save_csv(level2_df, savepath, savename)

print(f"samples: {len(level2_df)}")
print(f"TCRs:    {len(level2_df.drop_duplicates(subset=['beta.cdr3']))}")
print(f"HLAs:    {len(level2_df.drop_duplicates(subset=['hla.full.seq']))}")
print(f"HLAs:    {len(level2_df.drop_duplicates(subset=['hla.full.seq', 'hla.clip.seq', 'hla.short.seq']))}")
print(f"peptides:{level2_df['antigen.epitope'].nunique()}")

File /data/lujd/TCRdata/final/Hi-TpH-level-IIA.csv has been successfully saved.
samples: 78679
TCRs:    73553
HLAs:    88
HLAs:    88
peptides:1401


In [ ]:
for tag in level2_df['dataset.tag'].unique():
    tag_df = level2_df[level2_df['dataset.tag']==tag]
    print(tag,
          len(tag_df),
          tag_df['antigen.epitope'].nunique(),
          len(tag_df.drop_duplicates(subset=['beta.cdr3'])),
          )

iedb 33651 345 31708
vdjdb 38839 1004 36281
mcpas 6189 278 6059


---

### Level-III

- pep hla cdr3a cdr3b

In [30]:
level3_df = merge_df[['antigen.epitope', 'hla.allele', 'alpha.cdr3', 'beta.cdr3', 'dataset.tag']].drop_duplicates(
    subset=['antigen.epitope', 'hla.allele', 'alpha.cdr3', 'beta.cdr3'], ignore_index=True, keep='first')
print(len(level3_df))

level3_df = level3_df[level3_df['antigen.epitope']!='(UNK)']
level3_df = level3_df[level3_df['alpha.cdr3']!='(UNK)']
level3_df = level3_df[level3_df['beta.cdr3']!='(UNK)']
level3_df = level3_df.reset_index(drop=True)
print(f"number of dropna(pep,cdr3a,cdr3b) data: {len(level3_df)}")

level3_df = level3_df[level3_df['hla.allele']!='(UNK)']
level3_df = level3_df.reset_index(drop=True)
print(f"number of dropna(hla) data: {len(level3_df)}")

print(f"samples: {len(level3_df)}")
print(f"TCRs:    {len(level3_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3']))}")
print(f"peptides:{level3_df['antigen.epitope'].nunique()}")

171455
number of dropna(pep,cdr3a,cdr3b) data: 28740
number of dropna(hla) data: 28735
samples: 28735
TCRs:    26497
peptides:1150


In [31]:
target_hla_list = level3_df['hla.allele'].unique().tolist()
print(len(target_hla_list))
target_hla_list[:5]

85


['HLA-E*01:01', 'HLA-E*01:03', 'HLA-A*02:01', 'HLA-B*44:02', 'HLA-B*44:03']

In [32]:
import os
import json
dict_savepath = to_your_path + "TCRdata/final"
dict_savename = "Hi-TpH-hla_allele2seq.json"        # simple version for our dataset
with open(os.path.join(dict_savepath, dict_savename), "r") as f:
    seq_dict = json.load(f)

print(len(list(seq_dict.keys())))
list(seq_dict.keys())[:5]

115


['HLA-E*01:01', 'HLA-E*01:03', 'HLA-B*08', 'HLA-B*44:05', 'HLA-B*08:01']

In [33]:
print(set(target_hla_list) - set(seq_dict.keys()))
print(len(level3_df[level3_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]))

level3_df = level3_df[~level3_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]
level3_df = level3_df.reset_index(drop=True)
print(f"number of hla seq data: {len(level3_df)}")

{'HLA-B*12', 'HLA-A*24:01', 'HLA-A*08:01'}
144
number of hla seq data: 28591


In [34]:
level3_df['hla.full.seq'] = level3_df['hla.allele'].apply(lambda x: seq_dict[x]['full sequence'])
level3_df['hla.clip.seq'] = level3_df['hla.allele'].apply(lambda x: seq_dict[x]['clip sequence'])
level3_df['hla.short.seq'] = level3_df['hla.allele'].apply(lambda x: seq_dict[x]['short sequence'])

level3_df = level3_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.cdr3', 'beta.cdr3', 'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(level3_df))

print(len(level3_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'antigen.epitope', 'hla.full.seq'])))
print(len(level3_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'antigen.epitope', 'hla.clip.seq'])))
print(len(level3_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'antigen.epitope', 'hla.short.seq'])))

28262
28262
28255
28248


In [35]:
for c in level3_df:
    print(len(level3_df[level3_df[c]=="(UNK)"]))

0
0
0
0
0
0
0
0


In [36]:
savename = "Hi-TpH-level-III.csv"
save_csv(level3_df, savepath, savename)

print(f"samples: {len(level3_df)}")
print(f"TCRs:    {len(level3_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3']))}")
print(f"HLAs:    {len(level3_df.drop_duplicates(subset=['hla.full.seq']))}")
print(f"HLAs:    {len(level3_df.drop_duplicates(subset=['hla.full.seq', 'hla.clip.seq', 'hla.short.seq']))}")
print(f"peptides:{level3_df['antigen.epitope'].nunique()}")

File /data/lujd/TCRdata/final/Hi-TpH-level-III.csv has been successfully saved.
samples: 28262
TCRs:    26353
HLAs:    64
HLAs:    64
peptides:1148


In [37]:
for tag in level3_df['dataset.tag'].unique():
    tag_df = level3_df[level3_df['dataset.tag']==tag]
    print(tag,
          len(tag_df),
          tag_df['antigen.epitope'].nunique(),
          len(tag_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3'])),
          )

iedb 339 218 167
vdjdb 26608 828 24930
mcpas 1315 209 1259


---

### Level-IV

In [72]:
merge_df.columns

Index(['antigen.epitope', 'hla.allele', 'mhc.class', 'alpha.v', 'alpha.j',
       'alpha.cdr1', 'alpha.cdr2', 'alpha.cdr3', 'alpha.vseq', 'beta.d',
       'beta.j', 'beta.cdr1', 'beta.cdr2', 'beta.cdr3', 'beta.vseq',
       'dataset.tag', 'check', 'beta.v'],
      dtype='object')

In [73]:
cols = ['antigen.epitope', 'hla.allele', 
        'alpha.v', 'alpha.j', 'alpha.cdr3', 'alpha.vseq',
        'beta.v', 'beta.j', 'beta.cdr3', 'beta.vseq']

level4_df = merge_df[cols+['dataset.tag']].drop_duplicates(
    subset=cols, ignore_index=True, keep='first')
print(len(level4_df))

for col in cols:
    if col not in ['alpha.vseq', 'beta.vseq']:
        level4_df = level4_df[level4_df[col]!='(UNK)']
        level4_df = level4_df.reset_index(drop=True)
        print(f"number of dropna({col}) data: {len(level4_df)}")

print(f"samples: {len(level4_df)}")
print(f"TCRs:    {len(level4_df.drop_duplicates(subset=['alpha.v', 'alpha.j', 'alpha.cdr3', 'beta.v', 'beta.j', 'beta.cdr3']))}")
print(f"peptides:{level4_df['antigen.epitope'].nunique()}")

174419
number of dropna(antigen.epitope) data: 174419
number of dropna(hla.allele) data: 87749
number of dropna(alpha.v) data: 29448
number of dropna(alpha.j) data: 27681
number of dropna(alpha.cdr3) data: 27611
number of dropna(beta.v) data: 27591
number of dropna(beta.j) data: 27575
number of dropna(beta.cdr3) data: 27575
samples: 27575
TCRs:    25651
peptides:960


In [74]:
target_trgene_list = []
for col in ['alpha.v', 'alpha.j', 'beta.v', 'beta.j']:
    target_trgene_list.extend(level4_df[col].unique().tolist())

print(len(target_trgene_list))
target_trgene_list[:5]

369


['TRAV26-1*01', 'TRAV21*01', 'TRAV12-2*01', 'TRAV19*01', 'TRAV20*01']

In [75]:
import os
import json
dict_savepath = to_your_path + "TCRdata/collect"
dict_savename = "tcr_gene2seq.json"
with open(os.path.join(dict_savepath, dict_savename), "r") as f:
    gene2seq_dict = json.load(f)

all_gene_list = list(gene2seq_dict.keys())     # sorted
print(len(all_gene_list))
all_gene_list[:5]

350


['TRAV1-1*01', 'TRAV1-1*02', 'TRAV1-2*01', 'TRAV1-2*02', 'TRAV1-2*03']

In [76]:
seq_dict = {}
for target_trgene in target_trgene_list:
    info = {}
    for gene in all_gene_list:
        if (target_trgene in gene):
            """
            or (target_trgene.split('*')[0] in gene): 
            - 24 Not found -> 22
            - 184 Perfect -> 156
            """
            info['source gene'] = gene
            info['species'] = gene2seq_dict[gene]['species']
            info['amino acid sequence'] = gene2seq_dict[gene]['amino acid sequence']
            info['sequence length(aa)'] = gene2seq_dict[gene]['sequence length(aa)']
            
            if target_trgene==gene:
                print(f"Perfect: {target_trgene}")
                info['match source gene'] = "Perfect"
            else:
                print(f"Not perfect: {target_trgene} | {gene}")     # Missing "*01"
                info['match source gene'] = "Not perfect"
            seq_dict[target_trgene] = info
            break
    if target_trgene not in seq_dict.keys():
        info['source gene'] = '(UNK)'
        info['species'] = '(UNK)'
        info['amino acid sequence'] = '(UNK)'
        info['sequence length(aa)'] = '(UNK)'
        
        print(f"Not found: {target_trgene}")
        info['match source gene'] = "Not found"
        seq_dict[target_trgene] = info

Perfect: TRAV26-1*01
Perfect: TRAV21*01
Perfect: TRAV12-2*01
Perfect: TRAV19*01
Perfect: TRAV20*01
Perfect: TRAV5*01
Perfect: TRAV12-1*01
Perfect: TRAV3*01
Perfect: TRAV12-3*01
Perfect: TRAV39*01
Perfect: TRAV8-3*01
Perfect: TRAV22*01
Perfect: TRAV17*01
Perfect: TRAV35*01
Perfect: TRAV9-2*01
Perfect: TRAV27*01
Perfect: TRAV24*01
Perfect: TRAV30*01
Perfect: TRAV26-2*01
Perfect: TRAV1-2*01
Perfect: TRAV4*01
Perfect: TRAV13-1*01
Perfect: TRAV1-1*01
Perfect: TRAV8-1*01
Perfect: TRAV16*01
Perfect: TRAV6*01
Perfect: TRAV10*02
Perfect: TRAV29/DV5*01
Perfect: TRAV8-2*01
Perfect: TRAV8-6*02
Perfect: TRAV25*01
Perfect: TRAV1-2*03
Perfect: TRAV8-4*01
Perfect: TRAV2*01
Perfect: TRAV26-1*02
Perfect: TRAV38-2/DV8*01
Perfect: TRAV38-1*01
Perfect: TRAV13-2*01
Perfect: TRAV14/DV4*01
Perfect: TRAV10*01
Perfect: TRAV23/DV6*01
Perfect: TRAV36/DV7*01
Perfect: TRAV41*01
Perfect: TRAV8-6*01
Perfect: TRAV34*01
Perfect: TRAV14/DV4*02
Perfect: TRAV18*01
Perfect: TRAV40*01
Perfect: TRAV9-1*01
Not perfect: TRAV2 

In [77]:
savename = "Hi-TpH-tcr_gene2seq.json"

with open(os.path.join(savepath, savename), "w") as f:
    json.dump(seq_dict, f, indent=4)

print(F"File {os.path.join(savepath, savename)} has been successfully saved.")

File /data/lujd/TCRdata/final/Hi-TpH-tcr_gene2seq.json has been successfully saved.


In [78]:
for col in ['alpha.v', 'alpha.j', 'beta.v', 'beta.j']:
    level4_df[col+'.imgt.gene'] = level4_df[col].apply(lambda x: seq_dict[x]['source gene'])
    level4_df[col+'.imgt.seq'] = level4_df[col].apply(lambda x: seq_dict[x]['amino acid sequence'])

level4_df.head(2)

,antigen.epitope,hla.allele,alpha.v,alpha.j,alpha.cdr3,alpha.vseq,beta.v,beta.j,beta.cdr3,beta.vseq,dataset.tag,alpha.v.imgt.gene,alpha.v.imgt.seq,alpha.j.imgt.gene,alpha.j.imgt.seq,beta.v.imgt.gene,beta.v.imgt.seq,beta.j.imgt.gene,beta.j.imgt.seq
0,VMAPRTLIL,HLA-E*01:01,TRAV26-1*01,TRAJ37*01,IVVRSSNTGKLI,TTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIH...,TRBV14*01,TRBJ2-3*01,ASSQDRDTQY,GVTQFPSHSVIEKGQTVTLRCDPISGHDNLYWYRRVMGKEIKFLLH...,iedb,TRAV26-1*01,DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQY...,TRAJ37*01,GSGNTGKLIFGQGTTLQVKP,TRBV14*01,EAGVTQFPSHSVIEKGQTVTLRCDPISGHDNLYWYRRVMGKEIKFL...,TRBJ2-3*01,STDTQYFGPGTRLTVL
1,VMAPRTLIL,HLA-E*01:03,TRAV26-1*01,TRAJ37*01,IVVRSSNTGKLI,TTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIH...,TRBV14*01,TRBJ2-3*01,ASSQDRDTQY,GVTQFPSHSVIEKGQTVTLRCDPISGHDNLYWYRRVMGKEIKFLLH...,iedb,TRAV26-1*01,DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQY...,TRAJ37*01,GSGNTGKLIFGQGTTLQVKP,TRBV14*01,EAGVTQFPSHSVIEKGQTVTLRCDPISGHDNLYWYRRVMGKEIKFL...,TRBJ2-3*01,STDTQYFGPGTRLTVL


In [79]:
for col in ['alpha.v', 'alpha.j', 'beta.v', 'beta.j']:
    print(len(level4_df[level4_df[col+'.imgt.gene']=="(UNK)"]))

for col in ['alpha.v', 'alpha.j', 'beta.v', 'beta.j']:
    level4_df = level4_df[level4_df[col+'.imgt.gene']!='(UNK)']
    level4_df = level4_df.reset_index(drop=True)
    print(f"number of dropna({col+'.imgt.gene'}) data: {len(level4_df)}")

48
10
267
1
number of dropna(alpha.v.imgt.gene) data: 27527
number of dropna(alpha.j.imgt.gene) data: 27517
number of dropna(beta.v.imgt.gene) data: 27265
number of dropna(beta.j.imgt.gene) data: 27265


In [80]:
for c in level4_df:
    print(c, len(level4_df[level4_df[c]=="(UNK)"]))

antigen.epitope 0
hla.allele 0
alpha.v 0
alpha.j 0
alpha.cdr3 0
alpha.vseq 26955
beta.v 0
beta.j 0
beta.cdr3 0
beta.vseq 26955
dataset.tag 0
alpha.v.imgt.gene 0
alpha.v.imgt.seq 0
alpha.j.imgt.gene 0
alpha.j.imgt.seq 0
beta.v.imgt.gene 0
beta.v.imgt.seq 0
beta.j.imgt.gene 0
beta.j.imgt.seq 0


In [81]:
level4_df['alpha.vseq.reconstructed'] = level4_df['alpha.v.imgt.seq']+'-'+level4_df['alpha.cdr3']+'-'+level4_df['alpha.j.imgt.seq']
level4_df['beta.vseq.reconstructed'] = level4_df['beta.v.imgt.seq']+'-'+level4_df['beta.cdr3']+'-'+level4_df['beta.j.imgt.seq']

- example

In [82]:
cdr3 = level4_df['alpha.cdr3'].tolist()[10]
vseq = level4_df['alpha.vseq'].tolist()[10]
revseqs = level4_df['alpha.vseq.reconstructed'].tolist()[10]

print(level4_df['alpha.v'].tolist()[10], cdr3, '\n', vseq, '\n', revseqs)

TRAV12-2*01 AVTTDSWGKLQ 
 EVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMSIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVTTDSWGKLQFGAGTQVVVTP 
 QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVN-AVTTDSWGKLQ-TTDSWGKLQFGAGTQVVVTP


In [83]:
vgene_seq = revseqs.split('-')[0]
cdr3 = revseqs.split('-')[1]
jgene_seq = revseqs.split('-')[2]

# cut vgene seq, eg. CA: ..YL|CAVT
cdr3_start = cdr3[:2]
vgene_seq_end = vgene_seq.rfind(cdr3_start)                 # last position

# cut jgene seq, eg. LQ: GKLQ|FG..
cdr3_end = cdr3[-2:]
jgene_seq_start = jgene_seq.find(cdr3_end)+len(cdr3_end)    # first position + 2    

revseq = vgene_seq[:vgene_seq_end] + cdr3 + jgene_seq[jgene_seq_start:]
vseq in revseq, revseq

(False,
 'QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPELIMFIYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCAVTTDSWGKLQFGAGTQVVVTP')

iedb.vseq: ..AIASLNCTYSDRGSQSFFWYRQYSGKSPELIM**S**IYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCA..
- TRAV12-2*02 in imgt

imgt.vseq: ..AIASLNCTYSDRGSQSFFWYRQYSGKSPELIM**F**IYSNGDKEDGRFTAQLNKASQYVSLLIRDSQPSDSATYLCA..
- TRAV12-2*01

In [84]:
def reconstruct_vseq(data):
    vgene_seq = data.split('-')[0]
    cdr3 = data.split('-')[1]
    jgene_seq = data.split('-')[2]

    # cut vgene seq, eg. CA: ..YL|CAVT
    cdr3_start = cdr3[:2]
    vgene_seq_end = vgene_seq.rfind(cdr3_start)                         # last position

    # cut jgene seq, eg. LQ: GKLQ|FG..
    cdr3_end = cdr3[-2:]
    jgene_seq_start = jgene_seq.find(cdr3_end) + len(cdr3_end)          # first position + 2    

    if (vgene_seq_end != -1) & (jgene_seq_start != (-1+len(cdr3_end))): # + len(cdr3_end) !!!
        return vgene_seq[:vgene_seq_end] + cdr3 + jgene_seq[jgene_seq_start:]
    else:
        return '(UNK)'

revseqs = level4_df['alpha.vseq.reconstructed'].tolist()[700:710]
for seq in revseqs:
    print(reconstruct_vseq(seq))

KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTSLLLIQSSQREQTSGRLNASLDKSSGRSTLYIAASQPGDSATYLCAVHTQGGSEKLVFGKGTKLTVNP
QQQVKQSPQSLIVQKGGISIINCAYENTAFDYFPWYQQFPGKGPALLIAIRPDVSEKKEGRFTISFNKSAKQFSLHIMDSQPGDSATYFCAASRGSGAGSYQLTFGKGTKLSVIP
ILNVEQSPQSLHVQEGDSTNFTCSFPSSNFYALHWYRWETAKSPEALFVMTLNGDEKKKGRISATLNTKEGYSYLYIKGSQPEDSATYLCALNQAGTALIFGKGTTLSVSS
ILNVEQSPQSLHVQEGDSTNFTCSFPSSNFYALHWYRWETAKSPEALFVMTLNGDEKKKGRISATLNTKEGYSYLYIKGSQPEDSATYLCARNTGNQFYFGTGTSLTVIP
GQQVMQIPQYQHVQEGEDFTTYCNSSTTLSNIQWYKQRPGGHPVFLIQLVKSGEVKKQKRLTFQFGEAKKNSSLHITATQTTDVGTYFCAGFNDYKLSFGAGTTVTVRA
(UNK)
DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHATLRDTAVYYCIAGGSQGNLIFGKGTKLSVKP
DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHATLRDTAVYYCIVSDIQGAQKLVFGQGTRLTINP
DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHATLRDTAVYYCIAVSQGGSEKLVFGKGTKLTVNP
DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHATLRDTAVYYCIVRVGYTGF

In [85]:
revseqs

['KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTSLLLIQSSQREQTSGRLNASLDKSSGRSTLYIAASQPGDSATYLCAVR-CAVHTQGGSEKLVF-TQGGSEKLVFGKGTKLTVNP',
 'QQQVKQSPQSLIVQKGGISIINCAYENTAFDYFPWYQQFPGKGPALLIAIRPDVSEKKEGRFTISFNKSAKQFSLHIMDSQPGDSATYFCAAS-CAASRGSGAGSYQLTF-YSGAGSYQLTFGKGTKLSVIP',
 'ILNVEQSPQSLHVQEGDSTNFTCSFPSSNFYALHWYRWETAKSPEALFVMTLNGDEKKKGRISATLNTKEGYSYLYIKGSQPEDSATYLCAF-CALNQAGTALIF-NQAGTALIFGKGTTLSVSS',
 'ILNVEQSPQSLHVQEGDSTNFTCSFPSSNFYALHWYRWETAKSPEALFVMTLNGDEKKKGRISATLNTKEGYSYLYIKGSQPEDSATYLCAF-CARNTGNQFYF-NTGNQFYFGTGTSLTVIP',
 'GQQVMQIPQYQHVQEGEDFTTYCNSSTTLSNIQWYKQRPGGHPVFLIQLVKSGEVKKQKRLTFQFGEAKKNSSLHITATQTTDVGTYFCAG-CAGFNDYKLSF-SNDYKLSFGAGTTVTVRA',
 'DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHATLRDTAVYYCIVRV-CTRGYARYNFNKFYF-YNFNKFYFGSGTKLNVKP',
 'DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHATLRDTAVYYCIVRV-CIAGGSQGNLIF-NYGGSQGNLIFGKGTKLSVKP',
 'DAKTTQPPSMDCAEGRAANLPCNHSTISGNEYVYWYRQIHSQGPQYIIHGLKNNETNEMASLIITEDRKSSTLILPHA

In [86]:
revseqs = level4_df['beta.vseq.reconstructed'].tolist()[10000:10010]
for seq in revseqs:
    print(reconstruct_vseq(seq))

DSGVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQYYNGEERAKGNILERFSAQQFPDLHSELNLSSLELGDSALYFCASSVVQENTGELFFGEGSRLTVL
EAGVAQSPRYKIIEKRQSVAFWCNPISGHATLYWYQQILGQGPKLLIQFQNNGVVDDSQLPKDRFSAERLKGVDSTLKIQPAKLEDSAVYLCASSLARGTGELFFGEGSRLTVL
NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSARDRVLYEQYFGPGTRLTVT
DAGVTQSPTHLIKTRGQQVTLRCSPKSGHDTVSWYQQALGQGPQFIFQYYEEEERQRGNFPDRFSGHQFPNYSSELNVNALLLGDSALYLCASSFDSKAFFGQGTRLTVV
EPEVTQTPSHQVTQMGQEVILRCVPISNHLYFYWYRQILGQKVEFLVSFYNNEISEKSEIFDDQFSVERPDGSNFTLKIRSTKLEDSAMYFCATTVPLSYEQYFGPGTRLTVT
DTAVSQTPKYLVTQMGNDKSIKCEQNLGHDTMYWYKQDSKKFLKIMFSYNNKELIINETVPNRFSPKSPDKAHLNLHINSLELGDSAVYFCASSLGTGGSYEQYFGPGTRLTVT
DSGVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQYYNGEERAKGNILERFSAQQFPDLHSELNLSSLELGDSALYFCASSEGKGRDSNQPQHFGDGTRLSIL
EAQVTQNPRYLITVTGKKLTVTCSQNMNHEYMSWYRQDPGLGLRQIYYSMNVEVTDKGDVPEGYKVSRKEKRNFPLILESPSPNQTSLYFCASSLYSPLAYSWNEKLFFGSGTQLSVL
DSGVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQYYNGEERAKGNILERFSAQQFPDLHSELNLS

In [87]:
revseqs

['DSGVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQYYNGEERAKGNILERFSAQQFPDLHSELNLSSLELGDSALYFCASSV-CASSVVQENTGELFF-NTGELFFGEGSRLTVL',
 'EAGVAQSPRYKIIEKRQSVAFWCNPISGHATLYWYQQILGQGPKLLIQFQNNGVVDDSQLPKDRFSAERLKGVDSTLKIQPAKLEDSAVYLCASSL-CASSLARGTGELFF-NTGELFFGEGSRLTVL',
 'NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSE-CASSARDRVLYEQYF-SYEQYFGPGTRLTVT',
 'DAGVTQSPTHLIKTRGQQVTLRCSPKSGHDTVSWYQQALGQGPQFIFQYYEEEERQRGNFPDRFSGHQFPNYSSELNVNALLLGDSALYLCASSL-CASSFDSKAFF-NTEAFFGQGTRLTVV',
 'EPEVTQTPSHQVTQMGQEVILRCVPISNHLYFYWYRQILGQKVEFLVSFYNNEISEKSEIFDDQFSVERPDGSNFTLKIRSTKLEDSAMYFCASSE-CATTVPLSYEQYF-SYEQYFGPGTRLTVT',
 'DTAVSQTPKYLVTQMGNDKSIKCEQNLGHDTMYWYKQDSKKFLKIMFSYNNKELIINETVPNRFSPKSPDKAHLNLHINSLELGDSAVYFCASSQ-CASSLGTGGSYEQYF-SYEQYFGPGTRLTVT',
 'DSGVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQYYNGEERAKGNILERFSAQQFPDLHSELNLSSLELGDSALYFCASSV-CASSEGKGRDSNQPQHF-SNQPQHFGDGTRLSIL',
 'EAQVTQNPRYLITVTGKKLTVTCSQNMNHEYMSWYRQDPGLGLRQIYYSMNVEVTDKGDVPEGYKVSRKEKRNF

In [88]:
level4_df['alpha.vseq.reconstructed'] = level4_df['alpha.vseq.reconstructed'].apply(reconstruct_vseq)
level4_df['beta.vseq.reconstructed'] = level4_df['beta.vseq.reconstructed'].apply(reconstruct_vseq)

for col in ['alpha.vseq.reconstructed', 'beta.vseq.reconstructed']:
    print(len(level4_df[level4_df[col]=="(UNK)"]))

for col in ['alpha.vseq.reconstructed', 'beta.vseq.reconstructed']:
    level4_df = level4_df[level4_df[col]!='(UNK)']
    level4_df = level4_df.reset_index(drop=True)
    print(f"number of dropna({col}) data: {len(level4_df)}")

860
178
number of dropna(alpha.vseq.reconstructed) data: 26405
number of dropna(beta.vseq.reconstructed) data: 26243


---

In [89]:
import os
import json
dict_savepath = to_your_path + "TCRdata/final"
dict_savename = "Hi-TpH-tcr_gene2seq.json"        # simple version for our dataset
with open(os.path.join(dict_savepath, dict_savename), "r") as f:
    seq_dict = json.load(f)

print(len(list(seq_dict.keys())))
list(seq_dict.keys())[:5]

369


['TRAV26-1*01', 'TRAV21*01', 'TRAV12-2*01', 'TRAV19*01', 'TRAV20*01']

In [90]:
pf_level4_df = level4_df

for col in ['alpha.v', 'alpha.j', 'beta.v', 'beta.j']:
    pf_level4_df['check'] = pf_level4_df[col].apply(lambda x: seq_dict[x]['match source gene'])
    pf_level4_df = pf_level4_df[pf_level4_df['check']=='Perfect'].reset_index(drop=True)
    print(f"number of perfect({col}) data: {len(pf_level4_df)}")

pf_level4_df = pf_level4_df.drop_duplicates(ignore_index=True, keep='first')
print(len(pf_level4_df))

number of perfect(alpha.v) data: 25752
number of perfect(alpha.j) data: 25752
number of perfect(beta.v) data: 25740
number of perfect(beta.j) data: 25740
25740


- hla

In [91]:
target_hla_list = pf_level4_df['hla.allele'].unique().tolist()
print(len(target_hla_list))
target_hla_list[:5]

81


['HLA-E*01:01', 'HLA-E*01:03', 'HLA-A*02:01', 'HLA-B*44:02', 'HLA-B*44:03']

In [92]:
import os
import json
dict_savepath = to_your_path + "TCRdata/final"
dict_savename = "Hi-TpH-hla_allele2seq.json"        # simple version for our dataset
with open(os.path.join(dict_savepath, dict_savename), "r") as f:
    seq_dict = json.load(f)

print(len(list(seq_dict.keys())))
list(seq_dict.keys())[:5]

115


['HLA-E*01:01', 'HLA-E*01:03', 'HLA-B*08', 'HLA-B*44:05', 'HLA-B*08:01']

In [93]:
print(set(target_hla_list) - set(seq_dict.keys()))
print(len(pf_level4_df[pf_level4_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]))

pf_level4_df = pf_level4_df[~pf_level4_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]
pf_level4_df = pf_level4_df.reset_index(drop=True)
print(f"number of hla seq data: {len(pf_level4_df)}")

{'HLA-B*12', 'HLA-A*24:01', 'HLA-A*08:01'}
133
number of hla seq data: 25607


In [94]:
pf_level4_df['hla.full.seq'] = pf_level4_df['hla.allele'].apply(lambda x: seq_dict[x]['full sequence'])
pf_level4_df['hla.clip.seq'] = pf_level4_df['hla.allele'].apply(lambda x: seq_dict[x]['clip sequence'])
pf_level4_df['hla.short.seq'] = pf_level4_df['hla.allele'].apply(lambda x: seq_dict[x]['short sequence'])

pf_level4_df = pf_level4_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.cdr3', 'beta.cdr3', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(pf_level4_df))

print(len(pf_level4_df.drop_duplicates(subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed', 'antigen.epitope', 'hla.full.seq'])))

25555
25492


In [95]:
pf_level4_df = pf_level4_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(pf_level4_df))

25492


In [96]:
for col in ['alpha.v', 'alpha.j', 'beta.v', 'beta.j']:
    print(sum(pf_level4_df[col] == pf_level4_df[col+'.imgt.gene']))

25492
25492
25492
25492


In [97]:
pf_level4_df['alpha.vseq.length'] = pf_level4_df['alpha.vseq.reconstructed'].apply(lambda x: len(x))
pf_level4_df['beta.vseq.length'] = pf_level4_df['beta.vseq.reconstructed'].apply(lambda x: len(x))

pf_level4_df[['alpha.vseq.length', 'beta.vseq.length']].describe()

,alpha.vseq.length,beta.vseq.length
count,25492.000000,25492.000000
mean,112.272595,114.030990
std,3.137798,2.366564
min,41.000000,32.000000
25%,110.000000,113.000000
50%,112.000000,114.000000
75%,114.000000,115.000000
max,125.000000,127.000000


In [98]:
pf_level4_df = pf_level4_df[pf_level4_df['alpha.vseq.length']>=100]
print(len(pf_level4_df))

pf_level4_df = pf_level4_df[pf_level4_df['beta.vseq.length']>=100]
print(len(pf_level4_df))

pf_level4_df = pf_level4_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(pf_level4_df))

25467
25458
25458


In [99]:
cols = ['antigen.epitope', 'hla.allele', 
        'alpha.v', 'alpha.j', 'alpha.cdr3',
        'beta.v', 'beta.j', 'beta.cdr3',
        'alpha.vseq.reconstructed', 'beta.vseq.reconstructed', 
        'hla.full.seq', 'hla.clip.seq', 'hla.short.seq']
pf_level4_df = pf_level4_df[cols+['dataset.tag']]

for c in pf_level4_df:
        print(c, len(pf_level4_df[pf_level4_df[c]=="(UNK)"]))

antigen.epitope 0
hla.allele 0
alpha.v 0
alpha.j 0
alpha.cdr3 0
beta.v 0
beta.j 0
beta.cdr3 0
alpha.vseq.reconstructed 0
beta.vseq.reconstructed 0
hla.full.seq 0
hla.clip.seq 0
hla.short.seq 0
dataset.tag 0


In [100]:
savename = "Hi-TpH-level-IV-tmp.csv"
save_csv(pf_level4_df, savepath, savename)

print(f"samples: {len(pf_level4_df)}")
print(f"TCRs:    {len(pf_level4_df.drop_duplicates(subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"TCRs:    {len(pf_level4_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"TCRs:    {len(pf_level4_df.drop_duplicates(subset=['alpha.v', 'alpha.j', 'beta.v', 'beta.j', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"TCRs:    {len(pf_level4_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'alpha.v', 'alpha.j', 'beta.v', 'beta.j', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"HLAs:    {len(pf_level4_df.drop_duplicates(subset=['hla.full.seq']))}")
print(f"peptides:{pf_level4_df['antigen.epitope'].nunique()}")

File /data/lujd/TCRdata/final/Hi-TpH-level-IV-tmp.csv has been successfully saved.
samples: 25458
TCRs:    23767
TCRs:    23779
TCRs:    23768
TCRs:    23780
HLAs:    61
peptides:870


In [102]:
for tag in pf_level4_df['dataset.tag'].unique():
    tag_df = pf_level4_df[pf_level4_df['dataset.tag']==tag]
    print(tag,
          len(tag_df),
          tag_df['antigen.epitope'].nunique(),
          len(tag_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'alpha.v', 'alpha.j', 'beta.v', 'beta.j', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed'])),
          )

iedb 232 145 122
vdjdb 25161 758 23593
mcpas 65 3 65


---

- merge STAPLER train/test data

In [49]:
filename = "../otherpapers/stapler.csv"
stapler_df = pd.read_csv(datapath+filename, sep=',')
len(stapler_df)

4457

In [50]:
col_dict = {'epitope_aa': 'antigen.epitope',
            'hla_long': 'hla.allele',
            'TRAV_IMGT': 'alpha.v',
            'TRAJ_IMGT': 'alpha.j',
            'cdr3_alpha_aa': 'alpha.cdr3',
            'TRBV_IMGT': 'beta.v',
            'TRBJ_IMGT': 'beta.j',
            'cdr3_beta_aa': 'beta.cdr3',
            'full_seq_reconstruct_alpha_aa': 'alpha.vseq.reconstructed',
            'full_seq_reconstruct_beta_aa': 'beta.vseq.reconstructed'}
stapler_df = stapler_df.rename(columns=col_dict)
stapler_df = stapler_df[col_dict.values()]

In [51]:
stapler_df['hla.allele'] = stapler_df['hla.allele'].apply(lambda x: 'HLA-'+x)

target_hla_list = stapler_df['hla.allele'].unique().tolist()
print(len(target_hla_list))
target_hla_list

9


['HLA-A*02:01',
 'HLA-A*01:01',
 'HLA-A*24:02',
 'HLA-A*03:01',
 'HLA-B*35:01',
 'HLA-B*07:02',
 'HLA-B*18:01',
 'HLA-B*08:01',
 'HLA-B*44:05:01']

In [52]:
import os
import json
dict_savepath = to_your_path + "TCRdata/final"
dict_savename = "Hi-TpH-hla_allele2seq.json"        # simple version for our dataset
with open(os.path.join(dict_savepath, dict_savename), "r") as f:
    seq_dict = json.load(f)

print(len(list(seq_dict.keys())))
list(seq_dict.keys())[:5]

115


['HLA-E*01:01', 'HLA-E*01:03', 'HLA-B*08', 'HLA-B*44:05', 'HLA-B*08:01']

In [53]:
print(set(target_hla_list) - set(seq_dict.keys()))
print(len(stapler_df[stapler_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]))

stapler_df = stapler_df[~stapler_df['hla.allele'].isin(set(target_hla_list) - set(seq_dict.keys()))]
stapler_df = stapler_df.reset_index(drop=True)
print(f"number of hla seq data: {len(stapler_df)}")

set()
0
number of hla seq data: 4457


In [54]:
stapler_df['hla.full.seq'] = stapler_df['hla.allele'].apply(lambda x: seq_dict[x]['full sequence'])
stapler_df['hla.clip.seq'] = stapler_df['hla.allele'].apply(lambda x: seq_dict[x]['clip sequence'])
stapler_df['hla.short.seq'] = stapler_df['hla.allele'].apply(lambda x: seq_dict[x]['short sequence'])

stapler_df = stapler_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.cdr3', 'beta.cdr3', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(stapler_df))

print(len(stapler_df.drop_duplicates(subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed', 'antigen.epitope', 'hla.full.seq'])))

4457
4457


In [74]:
stapler_df['alpha.vseq.length'] = stapler_df['alpha.vseq.reconstructed'].apply(lambda x: len(x))
stapler_df['beta.vseq.length'] = stapler_df['beta.vseq.reconstructed'].apply(lambda x: len(x))

stapler_df[['alpha.vseq.length', 'beta.vseq.length']].describe()

,alpha.vseq.length,beta.vseq.length
count,4457.000000,4457.000000
mean,112.034777,114.075611
std,2.835702,2.065551
min,100.000000,107.000000
25%,110.000000,113.000000
50%,112.000000,114.000000
75%,114.000000,115.000000
max,121.000000,124.000000


- merge

In [99]:
stapler_df = stapler_df.drop(columns=['alpha.vseq.length', 'beta.vseq.length'])
level4_merge_df = pd.concat((pf_level4_df, stapler_df), axis=0).reset_index(drop=True)

level4_merge_df = level4_merge_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.cdr3', 'beta.cdr3', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(level4_merge_df))

26710


In [100]:
dup_df = level4_merge_df.duplicated(subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                    'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'], keep=False)
dup_df = level4_merge_df[dup_df]
dup_df

,antigen.epitope,hla.allele,alpha.v,alpha.j,alpha.cdr3,beta.v,beta.j,beta.cdr3,alpha.vseq.reconstructed,beta.vseq.reconstructed,hla.full.seq,hla.clip.seq,hla.short.seq
24,SLLMWITQV,HLA-A*02:01,TRAV21*01,TRAJ6*01,AVRPTSGGSYIPT,TRBV6-5*01,TRBJ2-2*01,ASSYVGNTGELF,KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTS...,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
25,SLLMWITQC,HLA-A*02:01,TRAV21*01,TRAJ6*01,AVRPTSGGSYIPT,TRBV6-5*01,TRBJ2-2*01,ASSYVGNTGELF,KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTS...,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
69,ILAKFLHWL,HLA-A*02:01,TRAV22*01,TRAJ40*01,AVDSATALPYGYI,TRBV6-5*01,TRBJ1-1*01,ASSYQGTEAF,GIQVEQSPPDLILQEGANSTLRCNFSDSVNNLQWFHQNPWGQLINL...,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
78,AAGIGILTV,HLA-A*02:01,TRAV35*01,TRAJ49*01,AGGTGNQFY,TRBV10-3*01,TRBJ1-5*01,AISEVGVGQPQH,GQQLNQSPQSMFIQEGEDVSMNCTSSSIFNTWLWYKQEPGEGPVLL...,DAGITQSPRHKVTETGTPVTLRCHQTENHRYMYWYRQDPGHGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
81,HSKKKCDEL,HLA-B*08:01,TRAV9-2*01,TRAJ43*01,ALSDPVNDMR,TRBV11-2*01,TRBJ1-5*01,ASSLRGRGDQPQH,GNSVTQMEGPVTLSEEAFLTINCTYTATGYPSLFWYVQYPGEGLQL...,EAGVAQSPRYKIIEKRQSVAFWCNPISGHATLYWYQQILGQGPKLL...,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFDTAMSRPGRGEPRF...,GSHSMRYFDTAMSRPGRGEPRFISVGYVDDTQFVRFDSDAASPREE...,YDSEYRNIFTNTDESNLYLSYNYYTWAVDAYTWY
159,YLQPRTFLL,HLA-A*02:01,TRAV12-2*01,TRAJ30*01,AVNRDDKII,TRBV7-9*01,TRBJ2-7*01,ASSPDIEQY,QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPEL...,DTGVSQNPRHKITKRGQNVTFRCDPISEHNRLYWYRQTLGQGPEFL...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
25802,YLQPRTFLL,HLA-A*02:01,TRAV12-2*01,TRAJ30*01,CAVNRDDKIIF,TRBV7-9*01,TRBJ2-7*01,CASSPDIEQYF,QKEVEQNSGPLSVPEGAIASLNCTYSDRGSQSFFWYRQYSGKSPEL...,DTGVSQNPRHKITKRGQNVTFRCDPISEHNRLYWYRQTLGQGPEFL...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
26666,SLLMWITQV,HLA-A*02:01,TRAV21*01,TRAJ6*01,CAVRPTSGGSYIPTF,TRBV6-5*01,TRBJ2-2*01,CASSYVGNTGELFF,KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTS...,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
26667,SLLMWITQC,HLA-A*02:01,TRAV21*01,TRAJ6*01,CAVRPTSGGSYIPTF,TRBV6-5*01,TRBJ2-2*01,CASSYVGNTGELFF,KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTS...,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY
26668,AAGIGILTV,HLA-A*02:01,TRAV35*01,TRAJ49*01,CAGGTGNQFYF,TRBV10-3*01,TRBJ1-5*01,CAISEVGVGQPQHF,GQQLNQSPQSMFIQEGEDVSMNCTSSSIFNTWLWYKQEPGEGPVLL...,DAGITQSPRHKVTETGTPVTLRCHQTENHRYMYWYRQDPGHGLRLI...,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY


In [101]:
level4_merge_df = level4_merge_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(level4_merge_df))

26704


In [102]:
dup_df = level4_merge_df.duplicated(subset=['alpha.cdr3', 'beta.cdr3', 'alpha.v', 'alpha.j', 'beta.v', 'beta.j',
                                    'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'], keep=False)
dup_df = level4_merge_df[dup_df]
dup_df

,antigen.epitope,hla.allele,alpha.v,alpha.j,alpha.cdr3,beta.v,beta.j,beta.cdr3,alpha.vseq.reconstructed,beta.vseq.reconstructed,hla.full.seq,hla.clip.seq,hla.short.seq


In [103]:
level4_merge_df.loc[5464, 'beta.vseq.reconstructed']

'DAGVTQSPTHLIKTRGQQVTLRCSPISGHKSVSWYQQVLGQGPQFIFQYYEKEERGRGNFPDRFSARQFPNYSSELNVNALLLGDSALYLCASSGPGGVDEQYFGPGTRLTVT'

In [104]:
level4_merge_df.loc[25612, 'beta.vseq.reconstructed']

'DAGVIQSPRHEVTEMGQEVTLRCKPISGHDYLFWYRQTMMRGLELLIYFNNNVPIDDSGMPEDRFSAKMPNASFSTLKIQPSEPRDSAVYFCASSVTSGGRETQYFGPGTRLLVL'

In [65]:
level4_merge_df.loc[5464, 'beta.vseq.reconstructed'] == level4_merge_df.loc[25612, 'beta.vseq.reconstructed']

False

In [105]:
level4_merge_df = level4_merge_df.drop_duplicates(ignore_index=True, keep='first',
                                      subset=['alpha.cdr3', 'beta.cdr3', 'alpha.v', 'alpha.j', 'beta.v', 'beta.j',
                                              'antigen.epitope', 'hla.full.seq', 'hla.clip.seq', 'hla.short.seq'])
print(len(level4_merge_df))

26704


In [106]:
for c in level4_merge_df:
    print(c, len(level4_merge_df[level4_merge_df[c]=="(UNK)"]))

antigen.epitope 0
hla.allele 0
alpha.v 0
alpha.j 0
alpha.cdr3 0
beta.v 0
beta.j 0
beta.cdr3 0
alpha.vseq.reconstructed 0
beta.vseq.reconstructed 0
hla.full.seq 0
hla.clip.seq 0
hla.short.seq 0


In [108]:
savename = "Hi-TpH-level-IV.csv"
save_csv(level4_merge_df, savepath, savename)

print(f"samples: {len(level4_merge_df)}")
print(f"TCRs:    {len(level4_merge_df.drop_duplicates(subset=['alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"TCRs:    {len(level4_merge_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"TCRs:    {len(level4_merge_df.drop_duplicates(subset=['alpha.v', 'alpha.j', 'beta.v', 'beta.j', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"TCRs:    {len(level4_merge_df.drop_duplicates(subset=['alpha.cdr3', 'beta.cdr3', 'alpha.v', 'alpha.j', 'beta.v', 'beta.j', 'alpha.vseq.reconstructed', 'beta.vseq.reconstructed']))}")
print(f"HLAs:    {len(level4_merge_df.drop_duplicates(subset=['hla.full.seq']))}")
print(f"peptides:{level4_merge_df['antigen.epitope'].nunique()}")

File /data/lujd/TCRdata/final/Hi-TpH-level-IV-wSTAPLER.csv has been successfully saved.
samples: 26704
TCRs:    24626
TCRs:    24638
TCRs:    24627
TCRs:    24639
HLAs:    61
peptides:927
